In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm


pd.options.display.max_rows = 35 
pd.options.display.max_columns = None

In [2]:
col_ignore = ['scheme_name', 'wpt_name', 'source_class', 'quality_group', 'quantity_group', 'recorded_by', 
             'extraction_type_group', 'extraction_type_class','management_group', 'payment_type', 'region_code',
              'district_code', 'subvillage', 'scheme_management', 'source_type', 'waterpoint_type_group', 
              'num_private', 'funder']

water_values = pd.read_csv('../../data/raw/WaterValuesOriginal.csv')
print('Original Shape', water_values.shape)
water_values = water_values[[i for i in water_values.columns if i not in col_ignore]]
water_labels = pd.read_csv('../../data/raw/WaterLabelsOriginal.csv')
water_labels['target'] = water_labels.status_group.values  #change target column
water_labels.drop('status_group', axis = 1, inplace = True) # drop the original column

water_values = pd.merge(water_values, water_labels, on = 'id') #merge target and features
water_values['date_recorded_date'] = pd.to_datetime(water_values.date_recorded)
water_values['year_recorded'] = pd.DatetimeIndex(water_values.date_recorded_date).year
#target distribution is roughly the same as the population distribution so we decided to drop them because there
#are not many entried and because the recorded was earlier than construction (all in 2004)

water_values = water_values[water_values.year_recorded >= 2011].reset_index(drop = True)
water_values = water_values[(water_values.source != 'unknown') & (water_values.water_quality != 'unknown') & 
                           (water_values.quantity != 'unknown')]
print('New Shape', water_values.shape)

Original Shape (59400, 40)
New Shape (57247, 25)


In [3]:
#fixing dates 
median_con = water_values[water_values.construction_year != 0].construction_year.median()
water_values.construction_year.replace(0, median_con, inplace = True)
cons_year = water_values.construction_year.values 
rec_date = water_values.date_recorded.values 
def get_time_since_built(cons, rec): 
    time_array = []
    for c, r in zip(cons, rec): 
        r = int(r.split('-')[0])
        c = int(c)
        diff = r -c
        time_array.append(diff)
    return time_array

t_array = get_time_since_built(cons_year, rec_date)
water_values['time_passed'] = t_array
water_values.drop(['construction_year', 'date_recorded_date', 'year_recorded', 'date_recorded'], 
                  axis = 1, inplace = True)


#made a new column for the number of years that have passed since construction.  
#Removed the recorded date and construction date
#for cells where construction date was 0, replaced it with the median construction date of the df

In [4]:
water_values.tail(2)

,id,amount_tsh,gps_height,installer,longitude,latitude,basin,region,lga,ward,population,public_meeting,permit,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,target,time_passed
59367,31282,0.0,0,Musa,35.861315,-6.378573,Rufiji,Dodoma,Chamwino,Mvumi Makulu,0,True,True,nira/tanira,vwc,never pay,soft,insufficient,shallow well,hand pump,functional,11
59368,26348,0.0,191,World,38.104048,-6.747464,Wami / Ruvu,Morogoro,Morogoro Rural,Ngerengere,150,True,True,nira/tanira,vwc,pay when scheme fails,salty,enough,shallow well,hand pump,functional,9


In [5]:
#changes binary true-false to 0-1
#chanegd the colums public meeting, permit to binary instead of boolean

water_values.public_meeting.fillna(False, inplace = True)
water_values.public_meeting = water_values.public_meeting.astype(int)
water_values.permit.fillna(False, inplace = True)
water_values.permit = water_values.permit.astype(int)
water_values.tail(2)

,id,amount_tsh,gps_height,installer,longitude,latitude,basin,region,lga,ward,population,public_meeting,permit,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,target,time_passed
59367,31282,0.0,0,Musa,35.861315,-6.378573,Rufiji,Dodoma,Chamwino,Mvumi Makulu,0,1,1,nira/tanira,vwc,never pay,soft,insufficient,shallow well,hand pump,functional,11
59368,26348,0.0,191,World,38.104048,-6.747464,Wami / Ruvu,Morogoro,Morogoro Rural,Ngerengere,150,1,1,nira/tanira,vwc,pay when scheme fails,salty,enough,shallow well,hand pump,functional,9


In [6]:
## Combining 'other' in extract 
##combining the types of extract that have 'other' into a single type 'other'
def fix_extract_other(x): 
    if 'other' in x: 
        return 'extract_other'
    else: 
        return x

water_values['extraction_type'] = water_values.extraction_type.map(fix_extract_other)
water_values.tail()

,id,amount_tsh,gps_height,installer,longitude,latitude,basin,region,lga,ward,population,public_meeting,permit,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,target,time_passed
59364,60739,10.0,1210,CES,37.169807,-3.253847,Pangani,Kilimanjaro,Hai,Masama Magharibi,125,1,1,gravity,water board,pay per bucket,soft,enough,spring,communal standpipe,functional,14
59365,27263,4700.0,1212,Cefa,35.249991,-9.070629,Rufiji,Iringa,Njombe,Ikondo,56,1,1,gravity,vwc,pay annually,soft,enough,river,communal standpipe,functional,15
59366,37057,0.0,0,NaN,34.017087,-8.750434,Rufiji,Mbeya,Mbarali,Chimala,0,1,0,swn 80,vwc,pay monthly,fluoride,enough,machine dbh,hand pump,functional,11
59367,31282,0.0,0,Musa,35.861315,-6.378573,Rufiji,Dodoma,Chamwino,Mvumi Makulu,0,1,1,nira/tanira,vwc,never pay,soft,insufficient,shallow well,hand pump,functional,11
59368,26348,0.0,191,World,38.104048,-6.747464,Wami / Ruvu,Morogoro,Morogoro Rural,Ngerengere,150,1,1,nira/tanira,vwc,pay when scheme fails,salty,enough,shallow well,hand pump,functional,9


In [7]:
#fixing population to replace value with mdeian population within its basin 
def fix_population_basin(df): 
    new_df = df.reset_index(drop = True)
    for idx in tqdm(range(len(new_df))): 
        pop = new_df.iloc[idx].population 
        if pop != 0: 
            continue 
        basin_value = new_df.iloc[idx].basin
        basin_median_pop = new_df[(new_df.basin == basin_value) & (new_df.population > 0)].population.median() 
        new_df.loc[idx, 'population'] = basin_median_pop
        
    return new_df     

water_values = fix_population_basin(water_values)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57247/57247 [02:18<00:00, 412.96it/s]


In [8]:
# find median latitude and longitude by region excluding the bad 
median_lat_longs = water_values.loc[water_values.longitude != 0].groupby('region').median()[['latitude', 'longitude']]
median_lat_longs

min_long = water_values.longitude.min()
# the same entries have a bad latitude value as well
bad_lat = water_values.loc[water_values.region == 'Mwanza'].latitude.mode()[0]
# checking that this is true: same 1593 entries all have long of zero
water_values.loc[water_values.latitude == bad_lat].longitude.value_counts()

# dict of group medians by region for lat and long
lat_dict = dict(median_lat_longs.latitude)
long_dict = dict(median_lat_longs.longitude)
long_dict

# make temp df to hold the correct values in the 1593 locations where incorrect
temp = water_values.loc[water_values.longitude == min_long]
temp['longitude'], temp['latitude'] = temp.region, temp.region
temp['longitude'] = temp.longitude.map(lambda x: long_dict[x])
temp['latitude'] = temp.latitude.map(lambda x: lat_dict[x])


# replace longitudes with group median by region
water_values.loc[water_values['longitude'] == min_long, 'longitude'] = temp['longitude']
water_values.iloc[temp.index]

# replace latitudes with group median by region
water_values.loc[water_values['latitude'] == bad_lat, 'latitude'] = temp['latitude']
water_values.iloc[temp.index]

c:\users\heeeb\anaconda3\envs\codeacademy\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\heeeb\anaconda3\envs\codeacademy\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\heeeb\anaconda3\envs\codeacademy\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,id,amount_tsh,gps_height,installer,longitude,latitude,basin,region,lga,ward,population,public_meeting,permit,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,target,time_passed
161,72678,0.0,0,WVT,33.449412,-3.487160,Lake Victoria,Shinyanga,Bariadi,Chinamili,320.0,0,0,gravity,parastatal,other,soft,seasonal,rainwater harvesting,communal standpipe,functional,13
170,56725,0.0,0,DWE,33.449412,-3.487160,Lake Victoria,Shinyanga,Bariadi,Bunamhala,320.0,0,0,extract_other,wug,unknown,soft,enough,shallow well,other,non functional,13
240,13042,0.0,0,DWE,33.080677,-2.626817,Lake Victoria,Mwanza,Magu,Malili,320.0,1,1,nira/tanira,vwc,never pay,soft,insufficient,shallow well,hand pump,functional needs repair,12
269,71487,0.0,0,DWE,33.080677,-2.626817,Lake Victoria,Mwanza,Magu,Shigala,320.0,1,1,swn 80,vwc,pay when scheme fails,salty,insufficient,shallow well,hand pump,functional,12
285,70379,0.0,0,Government,33.080677,-2.626817,Lake Victoria,Mwanza,Geita,Kalangalala,320.0,1,1,afridev,vwc,never pay,soft,insufficient,machine dbh,hand pump,functional,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57047,62177,0.0,0,DWE,33.449412,-3.487160,Lake Victoria,Shinyanga,Bariadi,Mhunze,320.0,1,0,nira/tanira,wug,other,soft,enough,shallow well,hand pump,functional needs repair,11
57066,3631,0.0,0,DWE,33.449412,-3.487160,Lake Victoria,Shinyanga,Bariadi,Kinang'weli,320.0,0,0,nira/tanira,wug,unknown,soft,enough,shallow well,hand pump,functional,13
57149,60843,0.0,0,DWE,33.449412,-3.487160,Lake Victoria,Shinyanga,Bariadi,Chinamili,320.0,1,0,nira/tanira,wug,never pay,soft,enough,shallow well,hand pump,functional needs repair,11
57176,748,0.0,0,World Vision,33.449412,-3.487160,Lake Victoria,Shinyanga,Bariadi,Mbita,320.0,0,0,nira/tanira,wug,unknown,soft,enough,shallow well,hand pump,functional,13


In [9]:
#aggregating values that have abandoned to their corresponding type

def fix_quality(x): 
    if 'abandoned' in x: 
        new_x = x.split(' ')[0]
        return new_x
    else: 
        return x

water_values.water_quality = water_values.water_quality.map(fix_quality)

In [10]:
water_values.to_csv('../../data/processed/WaterUpdated.csv', index = False)

In [11]:
# for i in water_values.water_quality.unique(): 
#     print(i)
#     print(water_values[water_values.water_quality == i].target.value_counts(), '\n')

# print(water_values[water_values.quantity == 'unknown'].shape)

# for reg in water_values.region.unique(): 
#     print(reg.upper())
#     unknown = water_values[(water_values.region == reg) & (water_values.quantity == 'unknown')]
#     reg = water_values[(water_values.region == reg) & (water_values.quantity != 'unknown')]
#     print(f'Unknown: {len(unknown)}\tRegular: {len(reg)}', '\n')
    
    
# water_values.quantity.unique()# water_values.payment_type.unique()
# water_values.payment_type.unique()
# water_values.source_class.unique()
#water_values[water_values.source != 'unknown']

# print('Source')
# for i in water_values.source.unique(): 
#     if 'unknown' in i: 
#         print(water_values[water_values.source==i].target.value_counts(), '\n')

# print('Water_Quality')
# for i in water_values.water_quality.unique(): 
#     if 'unknown' in i: 
#         print(water_values[water_values.water_quality==i].target.value_counts(), '\n')
        
# print('Quantity')
# for i in water_values.quantity.unique(): 
#     if 'unknown' in i: 
#         print(water_values[water_values.quantity==i].target.value_counts(), '\n')
        
# print('Payment')
# for i in water_values.payment.unique(): 
#     if 'unknown' in i: 
#         print(water_values[water_values.payment==i].target.value_counts(), '\n')